# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

# import libraries
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from joblib import dump, load
import pandas as pd
from sqlalchemy import create_engine

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from tempfile import mkdtemp

[nltk_data] Downloading package punkt to /home/flavio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/flavio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/flavio/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table(table_name='InsertTableName', con=engine)

category_names = list(set(df.columns.tolist()) - set(['id','message','original','genre']))

X = df['message'].values
Y = df[category_names].values

In [3]:
Y.shape

(26216, 36)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
cachedir = mkdtemp()
cachedir

'/tmp/tmplwx2ak65'

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1))),
    ],
    
    memory=cachedir
)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)


In [9]:
pipeline.fit(X_train, y_train)

Pipeline(memory='/tmp/tmplwx2ak65',
         steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f2b42a2f820>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
preds = pipeline.predict(X_test)

for idx, category in enumerate(category_names):
        print(f'results for {category}: \n', 
            classification_report(y_test[idx], preds[idx]))

results for offer: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.99        34
           1       1.00      0.50      0.67         2

    accuracy                           0.97        36
   macro avg       0.99      0.75      0.83        36
weighted avg       0.97      0.97      0.97        36

results for refugees: 
               precision    recall  f1-score   support

           0       0.82      1.00      0.90        28
           1       1.00      0.25      0.40         8

    accuracy                           0.83        36
   macro avg       0.91      0.62      0.65        36
weighted avg       0.86      0.83      0.79        36

results for direct_report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      1.00      1.00         3

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        #'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100, 150],
        'clf__estimator__min_samples_split': [2, 3, 4],
    }

cv = GridSearchCV(pipeline, n_jobs=-1, verbose=2, param_grid=parameters)

cv.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:   13.2s finished


GridSearchCV(estimator=Pipeline(memory='/tmp/tmplwx2ak65',
                                steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f2b42a2f820>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))]),
             n_jobs=-1,
             param_grid={'clf__estimator__min_samples_split': [2, 3, 4],
                         'clf__estimator__n_estimators': [50, 100, 150],
                         'tfidf__use_idf': (True, False),
                         'vect__max_df': (0.5, 0.75, 1.0),
                         'vect__ngram_range': ((1, 1), (1, 2))},
             verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
cv.best_params_

{'clf__estimator__min_samples_split': 2,
 'clf__estimator__n_estimators': 50,
 'tfidf__use_idf': True,
 'vect__max_df': 0.5,
 'vect__ngram_range': (1, 1)}

In [13]:
best_model = cv.best_estimator_

preds = best_model.predict(X_test)

for idx, category in enumerate(category_names):
        print(f'results for {category}: \n', 
            classification_report(y_test[idx], preds[idx]))

results for offer: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.99        34
           1       1.00      0.50      0.67         2

    accuracy                           0.97        36
   macro avg       0.99      0.75      0.83        36
weighted avg       0.97      0.97      0.97        36

results for refugees: 
               precision    recall  f1-score   support

           0       0.82      1.00      0.90        28
           1       1.00      0.25      0.40         8

    accuracy                           0.83        36
   macro avg       0.91      0.62      0.65        36
weighted avg       0.86      0.83      0.79        36

results for direct_report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      1.00      1.00         3

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [14]:
dump(best_model, 'disaster-model.pkl')

['disaster-model.pkl']